1. Load the GigE/10GigE camera PDF manual
2. Split the text into manageable chunks to avoid overly long prompts
3. Embed each chunk and upsert to the Pinecone index
4. Retrieve the most relevant chunks for a user question
5. Send the retrieved context to the LLM with the question


In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = PyPDFLoader("GigE (10GigE) Area Scan Camera_User's Manual_V1.0.0.pdf")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

document_list = text_splitter.split_documents(documents)
len(document_list)


In [2]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embedding = OpenAIEmbeddings(model='text-embedding-3-large')

In [ ]:
import os

from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

from config import PINECONE_INDEX_NAME

index_name = PINECONE_INDEX_NAME
pinecone_api_key = os.environ.get("PINECONE_API_KEY")
if not pinecone_api_key:
    raise ValueError("Set the PINECONE_API_KEY environment variable before running this notebook.")

pc = Pinecone(api_key=pinecone_api_key)

database = PineconeVectorStore.from_existing_index(index_name=index_name, embedding=embedding)
upsert_ids = database.add_documents(document_list)
upsert_ids[:3] if upsert_ids else upsert_ids


c:\Users\dlwnd\project\inflearn-llm-application\inflearn-llm-application\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\dlwnd\project\inflearn-llm-application\inflearn-llm-application\Lib\site-packages\langchain_pinecone\__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorst

In [4]:
"""from langchain_chroma import Chroma

database = Chroma.from_documents(documents=document_list, embedding=embedding, collection_name="chroma-tax", persist_directory="./chroma")
database = Chroma(collection_name="chroma-tax", persist_directory="./chroma", embedding_function=embedding)"""

'from langchain_chroma import Chroma\n\ndatabase = Chroma.from_documents(documents=document_list, embedding=embedding, collection_name="chroma-tax", persist_directory="./chroma")\ndatabase = Chroma(collection_name="chroma-tax", persist_directory="./chroma", embedding_function=embedding)'

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model = "gpt-4o")

In [6]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [7]:
from langchain.chains import RetrievalQA
retriever = database.as_retriever()
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever = retriever,
    chain_type_kwargs = {"prompt" : prompt}
)

In [8]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

dictionary = [
    "초보자 -> Beginner",
    "전문가 -> Expert",
    "구루 -> Guru",
    "프레임 속도 -> Frame Rate",
    "프레임 수 -> AcquisitionFrameCount",
]

prompt = ChatPromptTemplate.from_template(
    "사용자의 질문을 보고, 사전을 참고해 GigE 카메라 관련 용어를 통일해주세요.
"
    "변경할 필요가 없다고 판단되면 질문을 그대로 반환하세요.
"
    "사전: {dictionary}

질문: {question}"
)

dictionary_chain = prompt | llm | StrOutputParser()
camera_chain = {"query": dictionary_chain} | qa_chain


In [9]:
query = "프레임 레이트를 조정할 때 고려해야 하는 요소는 무엇인가요?"
dict_message = dictionary_chain.invoke({"question": query})
retriever.invoke(query)


[Document(id='07ac5bc0-6356-46b8-817e-b4120ba66292', metadata={'source': './tax_docs/tax_with_markdown.docx'}, page_content='② 제1항에 따른 원천징수의무자는 이자소득 또는 배당소득의 지급금액이 대통령령으로 정하는 금액 이하인 경우에는 제1항에 따른 원천징수영수증을 발급하지 아니할 수 있다. 다만, 제133조의2제1항에 따라 원천징수영수증을 발급하는 경우와 이자소득 또는 배당소득을 받는 자가 원천징수영수증의 발급을 요구하는 경우에는 제1항에 따라 원천징수영수증을 발급하거나 통지하여야 한다.\n\n[전문개정 2009. 12. 31.]\n\n\n\n제133조의2(채권 등에 대한 원천징수 특례) ① 거주자 또는 비거주자가 채권등의 발행법인으로부터 이자등을 지급받거나 해당 채권등을 발행법인 또는 대통령령으로 정하는 법인(이하 이 항에서 “발행법인등”이라 한다)에게 매도하는 경우 그 채권등의 발행일 또는 직전 원천징수일을 시기(始期)로 하고, 이자등의 지급일 등 또는 채권등의 매도일 등을 종기(終期)로 하여 대통령령으로 정하는 기간계산방법에 따른 원천징수기간의 이자등 상당액을 제16조에 따른 이자소득으로 보고, 해당 채권등의 발행법인등을 원천징수의무자로 하며, 이자등의 지급일 등 또는 채권등의 매도일 등 대통령령으로 정하는 날을 원천징수 하는 때로 하여 제127조부터 제133조까지, 제164조 및 제164조의2의 규정을 적용한다. <개정 2010. 12. 27., 2013. 1. 1.>\n\n② 제1항에 따른 이자등 상당액의 계산방법과 제46조제1항에 따른 환매조건부채권매매거래 등의 경우의 원천징수에 관하여 필요한 사항은 대통령령으로 정한다.\n\n[본조신설 2009. 12. 31.]\n\n\n\n제3관 근로소득에 대한 원천징수 <개정 2009. 12. 31.>\n\n\n\n제134조(근로소득에 대한 원천징수시기 및 방법) ① 원천징수의무자가 매월분의 근로소득을 지급할 때

In [10]:
dict_message

'연봉 7000만원인 거주자의 소득세는 얼마인가요?'

In [11]:
ai_message = camera_chain.invoke({"question": query})
ai_message


{'query': '연봉 7000만원인 거주자의 소득세는 얼마인가요?',
 'result': '연봉 7,000만원인 거주자의 소득세는 624만원 + (7,000만원 - 5,000만원) * 24퍼센트로 계산됩니다. 따라서 총 소득세는 624만원 + 480만원 = 1,104만원입니다.'}